<a href="https://colab.research.google.com/github/tarashagarwal/Analyzing-a-Global-Supply-Chain-Network/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sn
import time

In [185]:
#Conver MNIST image files to Tensors (#of Imhges, Height, Width and Color Channels)
transform = transforms.ToTensor()


In [186]:
#Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [187]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [188]:
#Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [189]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [190]:
# Create a small batch size for Images ... lets say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=10, shuffle=False)


In [191]:
#Define out CNN MOdel
#Describe convolution layer and what its doing (2 convolutional layer)
#This is just an
conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)

In [192]:
#Grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
    break

In [193]:
X_train.shape
torch.Size([1, 28, 28])

torch.Size([1, 28, 28])

In [194]:
x = X_train.view(1, 1, 28, 28)

In [195]:
x = F.relu(conv1(x))

In [196]:
x.shape#There is no paddign defined so some data is being dropped

torch.Size([1, 6, 26, 26])

In [197]:
#pass through the pooling layer
x = F.max_pool2d(x,2,2)

In [198]:
x.shape #26/2 = 13

torch.Size([1, 6, 13, 13])

In [199]:
# Do out second convolution layer
x = F.relu(conv2(x))

In [200]:
x.shape #we didnt set padding so we lose 2 pixels outside of the image

torch.Size([1, 16, 11, 11])

In [201]:
x = F.max_pool2d(x,2,2)

In [202]:
x.shape

torch.Size([1, 16, 5, 5])